<a href="https://colab.research.google.com/github/dfukagaw28/ColabNotebooks/blob/main/%E5%BD%A2%E6%85%8B%E7%B4%A0%E8%A7%A3%E6%9E%90%E5%99%A8%E3%81%AE%E9%80%9F%E5%BA%A6%E3%82%92%E6%AF%94%E8%BC%83%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 形態素解析器の速度を比較する

*  参考: [Python で分かち書き の速度比較 / janome, sudachi, ginza, mecab, fugashi, tinysegmenter #NLP - Qiita](https://qiita.com/kfjt/items/8b2f4a04e4befff18e19)
*  上記記事の時点（2020年6月頃）では MeCab が圧倒的に速く，次点が fugashi だった。
*  今回の実験結果では意外にも SudachiPy, MeCab, fugashi が並ぶ結果となった。
   *  これらの３つは実行時間に大きな差はなく，実行するたびに順位が微妙に入れ替わる。

*  バージョンの違いは以下の通り。

| package | qiita記事 2020年6月 | 今回の実験 |
| --- | --- | --- |
| Python | ? | 3.10.12 |
| janome | 0.3.10 | 0.5.0 |
| SudachiPy | 0.4.5 | 0.6.8 |
| Ginza | 3.1.2 | 5.2.0 |
| MeCab | 0.996.5 | 1.0.9 |
| fugashi | 0.2.3 | 1.3.2 |
| TinySegmenter | 0.4 | 0.4 |


In [1]:
!pip install janome
!pip install sudachipy sudachidict_core
!pip install ginza ja_ginza
!pip install mecab-python3
!pip install fugashi[unidic-lite]
!pip install tinysegmenter

In [2]:
!python --version

Python 3.10.12


In [3]:
!pip freeze

absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
alabaster==0.7.16
albucore==0.0.15
albumentations==1.4.15
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.18.0
asn1crypto==1.5.1
astropy==6.1.3
astropy-iers-data==0.2024.9.12.13.29.57
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.17.0
bigquery-magics==0.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.4.3
bqplot==0.12.43
branca==0.7.2
build==1.2.2
CacheControl==0.14.0
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
clarabel==0.9.0
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.19.0
cloudpickle==2.2.1
cmake==3.30.3
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5

In [4]:
import time

from janome.tokenizer import Tokenizer
from sudachipy import dictionary
import spacy
from MeCab import Tagger as mecab
from fugashi import Tagger as fugashi
import tinysegmenter

def use_janome(s, cache=None):
  t = Tokenizer() if cache is None else cache
  [token.surface for token in t.tokenize(s)]
  return t


def use_sudachi(s, cache=None):
  t = dictionary.Dictionary().create() if cache is None else cache
  [token.surface() for token in t.tokenize(s)]
  return t


def use_ginza(s, cache=None):
  t = spacy.load('ja_ginza') if cache is None else cache
  [token for token in t(s)]
  return t


def use_mecab(s, cache=None):
  t = mecab('-Owakati')
  [token for token in t.parse(s).split(' ')]
  return t


def use_fugashi(s, cache=None):
  t = fugashi('-Owakati')
  t(s)
  return t


def use_tinysegmenter(s, cache=None):
  t = tinysegmenter.TinySegmenter()
  t.tokenize(s)
  return t


def stopwatch(func, times=100):
  # 桃太郎 芥川龍之介 https://www.aozora.gr.jp/cards/000879/card100.html
  s = 'むかし、むかし、大むかし、ある深い山の奥に大きい桃《もも》の木が一本あった。大きいとだけではいい足りないかも知れない。この桃の枝は雲の上にひろがり、この桃の根は大地《だいち》の底の黄泉《よみ》の国にさえ及んでいた。何でも天地｜開闢《かいびゃく》の頃《ころ》おい、伊弉諾《いざなぎ》の尊《みこと》は黄最津平阪《よもつひらさか》に八《やっ》つの雷《いかずち》を却《しりぞ》けるため、桃の実《み》を礫《つぶて》に打ったという、――その神代《かみよ》の桃の実はこの木の枝になっていたのである。'
  time_s = time.perf_counter()
  cache = None
  for i in range(times):
    cache = func(s, cache)
  time_e = time.perf_counter()
  return time_e - time_s


def main():
  d = {
    'janome': stopwatch(use_janome),
    'sudachi': stopwatch(use_sudachi),
    'ginza': stopwatch(use_ginza),
    'mecab': stopwatch(use_mecab),
    'fugashi': stopwatch(use_fugashi),
    'tinysegmenter': stopwatch(use_tinysegmenter)
    }
  d = sorted(d.items(), key=lambda x:x[1])
  print('\n'.join(map(str, d)))

main()

('fugashi', 0.1610346339998614)
('mecab', 0.21631081400005314)
('sudachi', 0.24592323499996382)
('tinysegmenter', 0.4478832140000577)
('janome', 3.666879201000029)
('ginza', 16.65652347599996)
